In [50]:
%pip install tensorflow==2.7.0 tensorflow-gpu==2.7.0 opencv-python mediapipe scikit-learn sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\dangl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [51]:
import numpy as np
import cv2
import os
import time
import mediapipe as mp
from matplotlib import pyplot as plt

In [52]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [53]:
def mediapipe_detection(input_image, model):
    # Convert image from BGR to RGB
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    # Make prediction
    detection_results = model.process(input_image)
    # Convert image back to BGR
    input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)

    return input_image, detection_results

In [54]:
def draw_landmarks(input_image, detection_results):
    # Draw face connections
    mp_drawing.draw_landmarks(input_image, detection_results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)

In [55]:
def draw_styled_landmarks(input_image, detection_results):
    # Set drawing specifications for face connections
    face_points = mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1) 
    face_lines = mp_drawing.DrawingSpec(color=(255,255,0), thickness=1, circle_radius=1)

    # Draw face connections
    mp_drawing.draw_landmarks(input_image, detection_results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, face_points, face_lines)


In [56]:
# Extract keypoints into flattened arrays
def extract_keypoints (detection_results):

    face = np.array([[results.x, results.y, results.z]
    for results in detection_results.face_landmarks.landmark]).flatten() if detection_results.face_landmarks else np.zeros(468*3)

    return np.concatenate([face])

In [57]:
# Path to store exported np array
DATA_PATH = os.path.join('EmotionData')

# Gesture that will be detected 
emotions = np.array(['happy','sad','angry','neutral','surprised'])

# Number of sequences of data
n_frames = 30
# Length of each sequence will be 30 frames
length_frames = 5

In [58]:
for emotion in emotions :
    for frame in range(n_frames) :
        try :
            os.makedirs(os.path.join(DATA_PATH, emotion, str(frame)))
        except :
            pass


In [59]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for emotion in emotions:
        # Loop through sequences aka videos
        for video in range(n_frames):
            # Loop through video length aka sequence length
            for frame_num in range(length_frames):

                # Read feed
                ret, input_frame = cap.read()

                # Make detections
                input_image, results = mediapipe_detection(input_frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(input_image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(input_image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
                    cv2.putText(input_image, 'Collecting frames for {} Video Number {}'.format(emotion, video), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', input_image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(input_image, 'Collecting frames for {} Video Number {}'.format(emotion, video), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', input_image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, emotion, str(video), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):

                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [60]:
cv2.destroyAllWindows()

In [61]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [62]:
label_map = {label:num for num, label in enumerate(emotions)}

In [63]:
sequences, labels = [], []
for emotion in emotions:
    for frame in np.array(os.listdir(os.path.join(DATA_PATH, emotion))).astype(int):
        window = []
        for frame_num in range(length_frames):
            res = np.load(os.path.join(DATA_PATH, emotion, str(frame), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[emotion])

In [64]:
X = np.array(sequences)

In [65]:
y = to_categorical(labels).astype(int)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [68]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [69]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(5,1404)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(emotions.shape[0], activation='softmax'))

In [70]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [71]:
model.fit(X_train, y_train, epochs=800, callbacks=[tb_callback])

Epoch 1/800
5/5 [==============================] - 3s 17ms/step - loss: 1.6141 - categorical_accuracy: 0.2042
Epoch 2/800
5/5 [==============================] - 0s 15ms/step - loss: 1.6122 - categorical_accuracy: 0.1408
Epoch 3/800
5/5 [==============================] - 0s 17ms/step - loss: 1.6201 - categorical_accuracy: 0.2324
Epoch 4/800
5/5 [==============================] - 0s 19ms/step - loss: 1.6388 - categorical_accuracy: 0.1901
Epoch 5/800
5/5 [==============================] - 0s 18ms/step - loss: 1.6098 - categorical_accuracy: 0.2113
Epoch 6/800
5/5 [==============================] - 0s 15ms/step - loss: 1.6018 - categorical_accuracy: 0.2113
Epoch 7/800
5/5 [==============================] - 0s 41ms/step - loss: 1.5983 - categorical_accuracy: 0.2113
Epoch 8/800
5/5 [==============================] - 0s 17ms/step - loss: 1.5937 - categorical_accuracy: 0.2324
Epoch 9/800
5/5 [==============================] - 0s 16ms/step - loss: 1.5882 - categorical_accuracy: 0.2465
Epoch 10/8

In [72]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [73]:
yhat = model.predict(X_test)

In [74]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [75]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[6, 0],
        [0, 2]],

       [[4, 0],
        [0, 4]],

       [[6, 0],
        [0, 2]]], dtype=int64)

In [76]:
accuracy_score(ytrue, yhat)

1.0

In [77]:
from scipy import stats

In [78]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, input_frame = cap.read()

        # Make detections
        input_image, detection_results = mediapipe_detection(input_frame, holistic)
        print(detection_results)
        
        # Draw landmarks
        draw_styled_landmarks(input_image, detection_results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(detection_results)
        sequence.append(keypoints)
        sequence = sequence[-5:]
        
        if len(sequence) == 5:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(emotions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-5:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if emotions[np.argmax(res)] != sentence[-1]:
                            sentence.append(emotions[np.argmax(res)])
                    else:
                        sentence.append(emotions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            
        cv2.rectangle(input_image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(input_image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', input_image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
neutral
<class 'mediapipe.python.solution_base.SolutionOutputs'>
